In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import cPickle as pickle
import csv
import sys
from NN import AutoEncoder as AE 

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


### Turn Data into all binary

In [3]:
with open('Data.pkl', 'rb') as f:
            data = pickle.load(f)
            X = data['X']
            Y = data['Y']

print("Y.shape : {0}".format(Y.shape))
print("X.shape : {0}".format(X.shape))

Y.shape : (4209L,)
X.shape : (4209L, 376L)


In [4]:
maxes = np.max(X, axis = 0)[:8]
print maxes

mins = np.min(X, axis = 0)[:8]
print mins

[55 28 51  7  4 34 12 25]
[1 1 1 1 1 3 1 1]


In [5]:
one_hot_tables = []
for i,m in enumerate(maxes):
    current_table = np.zeros((X.shape[0], m))
    
    for j in range(X.shape[0]):
        current_table[j, X[j,i]-1] = 1.0
        
    one_hot_tables.append(current_table)
one_hot_tables.append(X[:,8:])
one_hot_data = np.concatenate(one_hot_tables, axis=1)
print one_hot_data.shape

(4209L, 584L)


In [6]:
with open('Data_OneHot.pkl', 'wb') as f:
    pickle.dump({"X":one_hot_data,"Y":Y}, f)

## Load Data

In [66]:
with open('Data_OneHot.pkl', 'rb') as f:
            data = pickle.load(f)
            X = data['X']
            Y = data['Y']


#Shuffle data :
data = np.concatenate([Y.reshape((Y.shape[0],1)),X], axis=1)
np.random.shuffle(data)

Y = data[:,0]
Y=Y.reshape((Y.shape[0]),1)
X = data[:,1:]

x_train = X[:4000,:]
y_train = Y[:4000]

x_valid = X[4000:,:]
y_valid = Y[4000:]


print("X_train shape : {0}".format(x_train.shape))
print("Y_train shape : {0}".format(y_train.shape))
print("X_valid.shape : {0}".format(x_valid.shape))
print("Y_valid shape : {0}".format(y_valid.shape))

X_train shape : (4000L, 584L)
Y_train shape : (4000L, 1L)
X_valid.shape : (209L, 584L)
Y_valid shape : (209L, 1L)


## Auto-encoder

In [71]:
n_units = [64, 16]
ae_non_linearty = 'sigmoid'
ae = AE.AutoEncoder(X.shape[1], n_units, ae_non_linearty, learning_rate=0.004, L2=0.001)

In [72]:
n_samples = x_train.shape[0]
n_dims = x_train.shape[1]
mb_size = 64
nb_iter_epochs = 500
max_epochs = 35
out_freq = 10
best_ae_encoder_states = ae.get_encoder_states()
best_valid_score = 1e100

train_costs = []
valid_costs = []

x_rep, x_rec, cst = ae.test(x_valid, x_valid)
print "Initial valid cost: %f" % cst
print

for epoch in range(max_epochs):
    epoch_train_costs = []
    
    for it in range(nb_iter_epochs): 
        rand_indices = np.random.choice(n_samples, mb_size)
        mb_x = x_train[rand_indices, :]
        x_rep, x_rec, cst = ae.train(mb_x, mb_x)
        epoch_train_costs.append(cst)
    
    train_costs.append(np.mean(epoch_train_costs))
    
    x_rep, x_rec, cst = ae.test(x_valid, x_valid)
    valid_costs.append(cst)
    
    if cst < best_valid_score:
        best_valid_score=cst
        best_ae_encoder_states = ae.get_encoder_states()
    
    if epoch % out_freq == 0:
        print "Epoch %d" % epoch
        print "Avg. train cost: %f" % train_costs[-1]
        print "Avg. valid cost: %f" % valid_costs[-1]
        print

print "Epoch %d" % epoch
print "Avg. train cost: %f" % train_costs[-1]
print "Avg. valid cost: %f" % valid_costs[-1]
print
print "Done"
print "Best epoch = %d" % np.argmin(valid_costs)
print "With train cost = %f" % train_costs[np.argmin(valid_costs)]
print "With valid cost = %f" % valid_costs[np.argmin(valid_costs)]

Initial valid cost: 146.512833

Epoch 0
Avg. train cost: 30.216009
Avg. valid cost: 29.617865

Epoch 10
Avg. train cost: 8.809736
Avg. valid cost: 10.396852

Epoch 20
Avg. train cost: 6.973104
Avg. valid cost: 9.318824

Epoch 30
Avg. train cost: 6.443729
Avg. valid cost: 9.473703

Epoch 34
Avg. train cost: 6.306573
Avg. valid cost: 9.499455

Done
Best epoch = 23
With train cost = 6.754955
With valid cost = 9.313946


### Regressor

In [77]:
r_units = [32, 16]
regressor_non_linearty = 'rectifier'
aer = AE.AutoEncoderRegressor(X.shape[1], n_units, r_units, ae_non_linearty, regressor_non_linearty,
                              learning_rate=0.004, L2=0.005)

#use pre-trained AE :
aer.set_encoder_states(best_ae_encoder_states)

In [79]:
n_samples = x_train.shape[0]
n_dims = x_train.shape[1]
mb_size = 64
nb_iter_epochs = 500
max_epochs = 100
out_freq = 10
best_valid_score = 1e100
recon_ratio = 0.1

#We want to monitor REGRESSION cost here.
train_costs = []
valid_costs = []

x_rep, x_rec, ae_cst, regress_cst, cst = aer.train(x_valid, x_valid, y_valid, recon_ratio)
print "Initial valid cost: %f" % regress_cst
print

for epoch in range(max_epochs):
    epoch_train_costs = []
    
    for it in range(nb_iter_epochs): 
        rand_indices = np.random.choice(n_samples, mb_size)
        mb_x = x_train[rand_indices, :]
        mb_y = y_train[rand_indices, :]
        x_rep, x_rec, ae_cst, regress_cst, cst = aer.train(mb_x, mb_x, mb_y, recon_ratio)
        epoch_train_costs.append(regress_cst)
    
    train_costs.append(np.mean(epoch_train_costs))
    
    x_rep, x_rec, ae_cst, regress_cst, cst = aer.train(x_valid, x_valid, y_valid, recon_ratio)
    valid_costs.append(regress_cst)
    
    if regress_cst < best_valid_score:
        best_valid_score=regress_cst
        
    if epoch % out_freq == 0:
        print "Epoch %d" % epoch
        print "Avg. train cost: %f" % train_costs[-1]
        print "Avg. valid cost: %f" % valid_costs[-1]
        print

print "Epoch %d" % epoch
print "Avg. train cost: %f" % train_costs[-1]
print "Avg. valid cost: %f" % valid_costs[-1]
print
print "Done"
print "Best epoch = %d" % np.argmin(valid_costs)
print "With train cost = %f" % train_costs[np.argmin(valid_costs)]
print "With valid cost = %f" % valid_costs[np.argmin(valid_costs)]

Initial valid cost: 44.803757

Epoch 0
Avg. train cost: 6.251073
Avg. valid cost: 49.004574

Epoch 10
Avg. train cost: 6.175247
Avg. valid cost: 47.205826

Epoch 20
Avg. train cost: 5.956792
Avg. valid cost: 47.554798

Epoch 30
Avg. train cost: 6.066935
Avg. valid cost: 45.903400

Epoch 40
Avg. train cost: 6.119670
Avg. valid cost: 44.293114

Epoch 50
Avg. train cost: 5.937460
Avg. valid cost: 41.971100

Epoch 60
Avg. train cost: 5.593970
Avg. valid cost: 40.896271

Epoch 70
Avg. train cost: 5.936595
Avg. valid cost: 38.857204

Epoch 80
Avg. train cost: 5.533383
Avg. valid cost: 37.910671

Epoch 90
Avg. train cost: 5.694654
Avg. valid cost: 39.418575

Epoch 99
Avg. train cost: 6.204894
Avg. valid cost: 36.989510

Done
Best epoch = 94
With train cost = 5.114531
With valid cost = 36.887135


In [38]:
# Some notes...

#TODO :
# Targets: 
#   Standardize targets + linear output
# Overfitting:
#   Try Dropout (we overfit a lot on the regression)
#   Try bigger recon_ratio
#   Try to add some noise?
# Go beyond MLP ?
#   Could we do better than let's say this setup+spearmint optimization ?

# AE:

# n_units = [64, 16], L2=0.001
#Best epoch = 23
#With train cost = 6.754955
#With valid cost = 9.313946


